In [4]:
from typing import Optional

import collections
from dm_control import suite as dm_suite
import dm_env
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from acme import specs
from acme import wrappers
from acme.agents.jax import d4pg
from acme.jax import experiments
from acme.utils import loggers

2023-09-20 06:44:26.770965: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ImportError: cannot import name 'specs' from 'acme' (/usr/local/lib/python3.10/site-packages/acme/__init__.py)

In [ ]:
def make_environment(seed: int) -> dm_env.Environment:
  environment = dm_suite.load('cartpole', 'balance')

  # Make the observations be a flat vector of all concatenated features.
  environment = wrappers.ConcatObservationWrapper(environment)

  # Wrap the environment so the expected continuous action spec is [-1, 1].
  # Note: this is a no-op on 'control' tasks.
  environment = wrappers.CanonicalSpecWrapper(environment, clip=True)

  # Make sure the environment outputs single-precision floats.
  environment = wrappers.SinglePrecisionWrapper(environment)

  return environment

In [ ]:
def network_factory(spec: specs.EnvironmentSpec) -> d4pg.D4PGNetworks:
  return d4pg.make_networks(
      spec,
      # These correspond to sizes of the hidden layers of an MLP.
      policy_layer_sizes=(256, 256),
      critic_layer_sizes=(256, 256),
  )

In [ ]:
d4pg_config = d4pg.D4PGConfig(learning_rate=3e-4, sigma=0.2)
d4pg_builder = d4pg.D4PGBuilder(d4pg_config)